In [12]:
import tenseal as ts
import utils

In [14]:
#Pour la clée publique
import os
from pathlib import Path

keys_dir = "Keys_manipulator"

if os.path.exists(keys_dir) and os.path.isdir(keys_dir):
    files = list(Path(keys_dir).glob('*'))
    
    if len(files) == 1:
        
        public_key_path = str(files[0])
        context = ts.context_from(utils.read_data(public_key_path))
    else:
        raise FileNotFoundError(f"Le répertoire {keys_dir} doit contenir exactement un fichier (contient {len(files)} fichiers)")
else:
    raise FileNotFoundError(f"Le répertoire {keys_dir} n'existe pas ou n'est pas un répertoire")


In [21]:

##   
# Si FEVER>7 et loss_of_taste>7 et difficulty_breathing >7
# On renvoie 1
# Sinon on renvoie 0 , sachant qu'on a pas accès directement aux données

#Détéction de fièrvre dépassant 8 , sigmoid
# Approximation simple de seuil : f(x) = (x - 8) * k
# Si x < 8 → f(x) < 0 → proche de 0
# Si x > 8 → f(x) > 0 → proche de 1
#sigmoid(x) = 0.5 + 0.197 * x - 0.004 * x^3
#from https://eprint.iacr.org/2018/462.pdf
fever_doc= "Data_received/fever_encrypted.txt"
fever = utils.read_data(fever_doc)
fever = ts.ckks_vector_from(context, fever)
difficulty_breathing_doc ="Data_received/difficulty_breathing_encrypted.txt"
difficulty_breathing=utils.read_data(difficulty_breathing_doc)
difficulty_breathing = ts.ckks_vector_from(context, difficulty_breathing)
loss_of_taste_doc = "Data_received/loss_of_taste_encrypted.txt"
loss_of_taste = utils.read_data(loss_of_taste_doc)
loss_of_taste = ts.ckks_vector_from(context, loss_of_taste)



seuil = 7
k = 0.4

def sigmoid_approx_from_symptom(symptom_vector):
    x = (symptom_vector - seuil) * k
    x_cube = x * x * x
    return x * 0.197 - x_cube * 0.004 + 0.5

sigmoid_fever = sigmoid_approx_from_symptom(fever)
sigmoid_loss_of_taste = sigmoid_approx_from_symptom(loss_of_taste)
sigmoid_difficulty_breathing = sigmoid_approx_from_symptom(difficulty_breathing)


combined_condition = sigmoid_fever * sigmoid_loss_of_taste * sigmoid_difficulty_breathing


utils.write_data("outputs_manipulator/fever_sigmoided_manipulated_encrypted.txt", combined_condition.serialize())

